# Agentic tool usage by local LLMs with LangChain
In this notebook I will create several simple examples of Agentic tool usage by local LLMs implemented with LangChain framework

## Some examples from LM-Studio docs

In [1]:
from openai import OpenAI

In [2]:
# check if the server is running and the model is available
! curl http://localhost:1234/v1/models

{
  "data": [
    {
      "id": "qwen3-30b-a3b-thinking-2507",
      "object": "model",
      "owned_by": "organization_owner"
    },
    {
      "id": "kimi-dev-72b",
      "object": "model",
      "owned_by": "organization_owner"
    },
    {
      "id": "kimi-dev-72b-dwq",
      "object": "model",
      "owned_by": "organization_owner"
    },
    {
      "id": "qwen3-coder-30b-a3b-instruct",
      "object": "model",
      "owned_by": "organization_owner"
    },
    {
      "id": "glm-4.5-air-mlx",
      "object": "model",
      "owned_by": "organization_owner"
    },
    {
      "id": "qwen3-30b-a3b-instruct-2507",
      "object": "model",
      "owned_by": "organization_owner"
    },
    {
      "id": "qwen3-32b",
      "object": "model",
      "owned_by": "organization_owner"
    },
    {
      "id": "gemma-3-27b-it-qat",
      "object": "model",
      "owned_by": "organization_owner"
    },
    {
      "id": "text-embedding-nomic-embed-text-v1.5",
      "object": "model",
      "

In [ ]:
# Point to the local server
client = OpenAI(base_url="http://127.0.0.1:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="qwen3-coder-30b-a3b-instruct",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="Greetings, I'm Qwen, a language model from Alibaba Cloud's Tongyi Lab. \nI'm here to assist you with various questions and tasks.\nWith great knowledge and a helpful heart,\nI aim to make your day a bit more sparkly. \nWhether you need help with writing, coding, or just want to chat,\nI'm ready to lend a hand and make things better.\nSo feel free to ask, I'm here to serve,\nWith wit and wisdom, I'll keep you entertained.", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)


In [ ]:
# Connect to LM Studio
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

# Define a simple function
def say_hello(name: str) -> str:
    print(f"Hello, {name}!")

# Tell the AI about our function
tools = [
    {
        "type": "function",
        "function": {
            "name": "say_hello",
            "description": "Says hello to someone",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The person's name"
                    }
                },
                "required": ["name"]
            }
        }
    }
]

# Ask the AI to use our function
response = client.chat.completions.create(
    model="qwen3-coder-30b-a3b-instruct",
    messages=[{"role": "user", "content": "Can you say hello to Bob the Builder?"}],
    tools=tools
)

# Get the name the AI wants to use a tool to say hello to
# (Assumes the AI has requested a tool call and that tool call is say_hello)
tool_call = response.choices[0].message.tool_calls[0]
name = eval(tool_call.function.arguments)["name"]

# Actually call the say_hello function
say_hello(name) # Prints: Hello, Bob the Builder!


## LangChain for LLM agent tooling
LangChain for building agent tooling with your local LLM served by LM Studio via the OpenAI-compatible API. Here's a step-by-step guide:

__📊 Visualization of the model reasoning steps__

User Query: "What is 15*4+2?"
│
├─ Tool Descriptions Check:
│   ├─ "Useful for math problems" → MATCH ✅
│   └─ "Weather tool" → No match ❌
│
└─ Decision:
   ├─ Tool Use Priority: HIGH (matched tool exists)
      └─ Execute calculator
          Input: "15 * 4 + 2"
          Output: 62
   └─ Pre-training Knowledge: NOT USED

---

User Query: "What is photosynthesis?"
│
├─ Tool Descriptions Check:
│   ├─ "Math tool" → No match ❌
│   └─ "Science tool" → No match ❌ (if not defined)
│
└─ Decision:
   ├─ Tool Use Priority: LOW (no matches)
      └─ Skip tools
   └─ Pre-training Knowledge USED:
       Final Answer: "Photosynthesis is how plants..."


### Imports

In [143]:
#from langchain.chat_models import ChatOpenAI# OLD (pre-v0.2)
from langchain_community.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, Tool, AgentExecutor, create_react_agent, create_openai_tools_agent
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain_core.tools import tool
from langchain import hub
import requests
from bs4 import BeautifulSoup
import PyPDF2
from io import BytesIO
import os
from pathlib import Path

In [272]:
my_llms = {'qwen3-coder': 'qwen3-coder-30b-a3b-instruct', 
           'qwen3-thinking': 'qwen3-30b-a3b-thinking-2507',
           'glm-4.5-air': 'glm-4.5-air-mlx',
           'gpt-oss': 'gpt-oss-120b',
           }

In [ ]:
# separatly test the custom tool

def calculator(expression: str) -> float:
    expr = expression.strip('"\'')
    return eval(expr)

calculate("15 * 4 + 2")

### Method 1 - with LangChain Tool object

In [ ]:
# LM Studio settings
BASE_URL = "http://localhost:1234/v1"
API_KEY = "not-needed"  # Dummy key since LM Studio doesn't require auth

# Initialize the LLM
llm = ChatOpenAI(
    model_name="glm-4.5-air-mlx",  # Replace with your actual model name
    temperature=0.7,
    openai_api_key=API_KEY,
    openai_api_base=BASE_URL
)

def calculate(expression: str) -> float:
    """Evaluate a mathematical expression."""
    # Remove surrounding quotes if present
    stripped_expr = expression.strip('"\'')
    return eval(stripped_expr)


# Wrap the function in LangChain's Tool format
tools = [
    Tool(
        name="calculator",
        func=calculate,
        description="Useful for when you need to solve math problems. Input should be a mathematical expression as a string, like '15 * 4 + 2'"
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  # Shows reasoning process
)

response = agent.run('''What is 15 * 4 + 2?''')
print(response)  # Expected: "62"




> Entering new AgentExecutor chain...

<think>The human is asking for the result of a simple mathematical expression: "15 * 4 + 2". This is straightforward arithmetic that I can compute using the calculator tool.

I need to provide a JSON blob with:
- action: "calculator" (since I need to perform a calculation)
- action_input: the mathematical expression as a string, which is "15 * 4 + 2"

Let me create this JSON blob:</think>Action:
```
{
  "action": "calculator",
  "action_input": "15 * 4 + 2"
}
```
Observation: 62
Thought:
<think>The calculator tool has returned a result of 62 for the expression "15 * 4 + 2". Let me verify this calculation:

15 * 4 = 60
60 + 2 = 62

The result is correct. Now I need to provide a final answer using the "Final Answer" action in the JSON blob format.</think>Action:
```
{
  "action": "Final Answer",
  "action_input": "The result of 15 * 4 + 2 is 62."
}
```

> Finished chain.
The result of 15 * 4 + 2 is 62.


In [ ]:
# simple check to see if the server is running and the model is responsive
print(llm.invoke("Hello").content)

### Method 2 - with LangChain @tool decorator

Your function's docstring becomes the __tool's description__.

When the agent decides to use a tool-> It calls calculator("input"). The LLM reads the docstring to understand:

What the tool does, What input format it expects, When to use it

In [ ]:
# LM Studio settings
BASE_URL = "http://localhost:1234/v1"
API_KEY = "not-needed"  # Dummy key since LM Studio doesn't require auth

# Initialize the LLM
llm = ChatOpenAI(
    model_name="glm-4.5-air-mlx",  # Replace with your actual model name
    temperature=0.7,
    openai_api_key=API_KEY,
    openai_api_base=BASE_URL
)

# Tool definition using decorator (this is the complete tool)
@tool
def calculator(expression: str) -> float:
    """Useful for when you need to solve math problems. Input should be a mathematical expression as a string, like '15 * 4 + 2'"""
    expr = expression.strip('"\'')
    return eval(expr)

# Initialize agent (direct tool assignment)
agent = initialize_agent(
    tools=[calculator],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # More robust parsing
    verbose=True,
    handle_parsing_errors=True  # Graceful error handling
)

response = agent.run("What is 15 * 4 + 2?")
print(response)  # Expected: "62"


**Which to choose?**

__Method 1 (Tool object):__ Better for complex tools requiring extra configuration

__Method 2 (@tool decorator):__ More concise and modern, recommended for simpler tools

In [ ]:
# Non-tool usage example

response = agent.run("Are Pandas is an endangered species or a Python library")
print(response)



> Entering new AgentExecutor chain...

<think>The human is asking whether "Pandas" refers to an endangered species or a Python library. This requires factual knowledge about both topics.

Pandas (the animal) are indeed an endangered species of bear native to China, known for their distinctive black and white coloring.

Pandas (the library) is also a very popular Python library used for data analysis and manipulation. It provides fast, flexible, and expressive data structures designed to make working with relational or labeled data both easy and intuitive.

The question is asking me to clarify which one they're referring to, but since both are valid interpretations of "Pandas," I should provide information about both concepts rather than making a definitive choice. This doesn't require the use of any calculator or tools, so I can respond directly with factual information about both pandas (the animal) and pandas (the Python library).</think>
```
{
  "action": "Final Answer",
  "action

### Example with more custom tools for LLM agent

__What is .invoke()?__
In LangChain, when you create a tool with @tool, it becomes a Tool object that has an .invoke() method instead of being called directly like a regular Python function.

This is because LangChain tools need to handle:

Input validation, Error handling, Logging, Integration with the agent system

In [ ]:
@tool
def browse_and_scrape(url: str) -> str:
    """
    Opens a webpage URL and extracts main text content.
    
    Args:
        url: The webpage URL to open (e.g., "https://example.com")
    
    Returns:
        Main text content of the webpage
    
    Example usage:
        browse_and_scrape("https://www.example.com")
    """
    try:
        # Set headers to mimic a browser
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Remove script and style elements
        for element in soup(["script", "style"]):
            element.decompose()
            
        # Get main text (focus on body content)
        if soup.body:
            text = ' '.join(soup.body.stripped_strings)
        else:
            text = ' '.join(soup.stripped_strings)
        print(f'Number of characters in text: {len(text)}')

        return f"Content from {url}:\n\n{text[:2000]}..."  # Truncate for brevity/lower token consumption
        
    except Exception as e:
        return f"Error accessing {url}: {str(e)}"


In [ ]:
print(browse_and_scrape.invoke('https://en.wikipedia.org/wiki/Patulin'))

Number of characters in text: 14121
Content from https://en.wikipedia.org/wiki/Patulin:

Jump to content Main menu Main menu move to sidebar hide Navigation Main page Contents Current events Random article About Wikipedia Contact us Contribute Help Learn to edit Community portal Recent changes Upload file Special pages Search Search Appearance Donate Create account Log in Personal tools Donate Create account Log in Pages for logged out editors learn more Contributions Talk Contents move to sidebar hide (Top) 1 Biosynthesis, synthesis, and reactivity 2 Uses 3 Sources of exposure 4 Toxicity Toggle Toxicity subsection 4.1 Acute 4.2 Subacute 4.3 Genotoxicity 4.4 Reproduction studies 4.5 Immunotoxicity 4.6 Human health 5 Risk management and regulations 6 References 7 External links Toggle the table of contents Patulin 23 languages العربية Català Čeština Deutsch Español Français 한국어 Italiano Latina Latviešu Magyar Nederlands 日本語 Polski Português Română Русский Српски / srpski Srpskohrvatski 

In [ ]:
@tool
def open_pdf(file_path: str) -> str:
    """
    Opens a PDF file specified by the user and extracts its text content.

    Args:
        file_path: Full path to the PDF file (e.g., "/path/to/document.pdf")
    
    Returns:
        Text content of the PDF file
    
    Example usage:
        open_pdf("/Users/user/documents/report.pdf")
    """
    try:
        # Check if file exists
        if not os.path.exists(file_path):
            return f"Error: File '{file_path}' does not exist"
            
        with open(file_path, 'rb') as f:
            # Create PDF reader
            pdf_reader = PyPDF2.PdfReader(f)
            
            # Extract text from each page
            text_parts = []
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text_parts.append(f"--- Page {page_num + 1} ---\n{page.extract_text()}")
                
            return f"PDF Content from {file_path}:\n\n" + "\n".join(text_parts)
            
    except Exception as e:
        return f"Error reading PDF '{file_path}': {str(e)}"


In [ ]:
print(open_pdf.invoke(('/Users/danid/Downloads/DARPA_grants_submission/DARPA-PS-25-02-BTO-01.pdf')))

__Usage with LangChain LLM agent__

In [ ]:
# Initialize your LLM (replace with your LM Studio config)
llm = ChatOpenAI(
    model_name="glm-4.5-air-mlx",
    temperature=0.7,
    openai_api_key=API_KEY,
    openai_api_base=BASE_URL,
    max_tokens=4096,
)

# Create agent with custom tools
agent = initialize_agent(
    tools=[browse_and_scrape, open_pdf],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


# Example queries:
agent.run('''Summarize the following PDF document at path: /Users/danid/Downloads/DARPA_grants_submission/DARPA-PS-25-02-BTO-01.pdf
           What are the main ideas behind the text? what is it about?
          ''')




> Entering new AgentExecutor chain...

<think>The human wants me to summarize a PDF document located at "/Users/danid/Downloads/DARPA_grants_submission/DARPA-PS-25-02-BTO-01.pdf". They specifically want to know the main ideas and what the document is about.

To do this, I'll need to use the open_pdf tool to read and extract the text content from the specified PDF file. Then I'll analyze that content to provide a summary of the main ideas and purpose of the document.

Let me call the open_pdf tool with the provided file path.</think>
I'll help you summarize the PDF document. Let me first open and read the content of the file.
```
{
  "action": "open_pdf",
  "action_input": "/Users/danid/Downloads/DARPA_grants_submission/DARPA-PS-25-02-BTO-01.pdf"
}
```
Observation: PDF Content from /Users/danid/Downloads/DARPA_grants_submission/DARPA-PS-25-02-BTO-01.pdf:

--- Page 1 ---
Special Topic: Ag x BTO
BACKGROUND:
The Defense Advanced Research Projects Agency (DARPA) is looking for innovative 

"This document is a DARPA (Defense Advanced Research Projects Agency) solicitation notice for Special Topic: Ag x BTO (Agriculture × Biological Technologies Office). It's a call for proposals to develop innovative solutions for defending agriculture against both natural and man-made threats to enhance national security.\n\nMain ideas include:\n1. DARPA's Biological Technologies Office seeks to catalyze innovation in agricultural defense\n2. The solicitation focuses on five key technical areas:\n   - Early warning systems for chemical/biological threat surveillance\n   - Rapid-response agricultural countermeasures against threats\n   - Massively accelerated crop engineering for long-term solutions\n   - Integrated threat prediction and modeling systems\n   - Novel methods to detect human intervention and attribute provenance\n3. This is a limited submission period (June 1 - August 31, 2025)\n4. Selected submitters may be invited to attend a workshop and Pitch Day\n5. DARPA aims to encou

In [ ]:
# Initialize your LLM (replace with your LM Studio config)
llm = ChatOpenAI(
    model_name="glm-4.5-air-mlx",
    temperature=0.7,
    openai_api_key=API_KEY,
    openai_api_base=BASE_URL,
    max_tokens=4096,
)

# Create agent with custom tools
agent = initialize_agent(
    tools=[browse_and_scrape, open_pdf],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Example queries:
agent.run('''Open this URL page https://en.wikipedia.org/wiki/Patulin
           Browse and do basic scraping and find the IUPAC name of Patulin''')




> Entering new AgentExecutor chain...

<think>The human wants me to open a specific Wikipedia page about Patulin and find its IUPAC name. This is a straightforward request for web scraping information from a specific URL. I'll use the browse_and_scrape tool to access this page and extract the information.

The URL is: https://en.wikipedia.org/wiki/Patulin

I'll use this exact URL as the action input for the browse_and_scrape tool.</think>Action:
```
{
  "action": "browse_and_scrape",
  "action_input": "https://en.wikipedia.org/wiki/Patulin"
}
```Number of characters in text: 14121

Observation: Content from https://en.wikipedia.org/wiki/Patulin:

Jump to content Main menu Main menu move to sidebar hide Navigation Main page Contents Current events Random article About Wikipedia Contact us Contribute Help Learn to edit Community portal Recent changes Upload file Special pages Search Search Appearance Donate Create account Log in Personal tools Donate Create account Log in Pages for log

'The IUPAC name of Patulin is: 4-hydroxy-4H-furo[3,2-c]pyran-2(6H)-one'

### More modern way to build LLM agent with LangChain + Chat

💡 Why create_react_agent() is better than initialize_agent():
* Handles tool returns correctly (no more "Tool returned invalid format" errors)
* Uses structured prompts matching the ReAct framework (e.g., Thought, Action, Observation)
* Works seamlessly with the new @tool decorator
* Officially recommended by LangChain for new projects

__LangChain v0.2 split the agent from its execution logic:__

create_react_agent() = Just the agent (a stateless prompt + tool handler)

AgentExecutor = The engine that runs the agent (handles loops, tool calls, error handling)

agent.run() was a convenience method in v0.1 that combined both steps.

Another point, LangChain v0.2 requires explicit prompts for agents (unlike v0.1). `ReActTextPrompt()` is the default React prompt you need.


__Update:__ 
LM Studio provides an OpenAI-compatible API server for running LLMs and returns OpenAI-like response objects (with **native tool calling**). 
This means that applications and frameworks designed to interact with OpenAI's API can be configured to communicate with LM Studio endpoint wuth your local LLM. 

To leverage this feature I can use `create_openai_tools_agent()` instead of `create_react_agent()` for agent creation.

__When to use the different agent types:__

Use initialize_agent if:
* You're using LangChain < v0.1
* Need simple zero-shot agents without complex tools

Use create_react_agent if:
* Your LLM doesn't support function calling
* You need explicit "Thought/Action" logging

Use create_openai_tools_agent for:
* LM Studio/OpenAI-style APIs ✅
* Complex tool interactions ✅
* Built-in chat history ✅

__Additional difference between create_react_agent() vs. create_openai_tools_agent():__

* create_react_agent() natively support tools usage, it is explicitly defined in its PromptTemplate, but it DOES NOT support "chat-history" - for single prompts uses only

* create_openai_tools_agent() does not explicitly define tools usage in its ChatPromptTemplate, and this capability depends on the underlying model, but it DOES support "chat-history"

In [151]:
prompt = hub.pull("hwchase17/react")  # for create_react_agent()
prompt

/opt/homebrew/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [ ]:
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt

/opt/homebrew/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [ ]:
# 🚀 Full Working Agent Example (v0.2+ Style)
BASE_URL = "http://localhost:1234/v1"
API_KEY = "not-needed"  # Dummy key since LM Studio doesn't require auth

llm = ChatOpenAI(
    model_name="glm-4.5-air-mlx",
    temperature=0.7,
    openai_api_key=API_KEY,
    openai_api_base=BASE_URL
)

prompt = hub.pull("hwchase17/react")  # for create_react_agent()

# 3. Create the agent (REACT framework)
agent = create_react_agent(
    llm=llm,
    tools=[browse_and_scrape, open_pdf],
    prompt=prompt,
)

# Run the agent, the "new way" with AgentExecutor. No change in usage pattern! just pass the input as a dict instead of a string. This is required to work with LangChain's new architecture.
agent_executor = AgentExecutor(agent=agent, tools=[open_pdf],
                               verbose=True,
                               handle_parsing_errors=True) # Avoids crashing on errors)

response = agent_executor.invoke({"input": 
                                '''Summarize the following PDF document at path:
                                /Users/danid/Downloads/DARPA_grants_submission/DARPA-PS-25-02-BTO-01.pdf
                                What are the main ideas behind the text? what is it about?'''})
print(response["output"])


/opt/homebrew/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...

<think>I need to summarize a PDF document. The user has provided the file path "/Users/danid/Downloads/DARPA_grants_submission/DARPA-PS-25-02-BTO-01.pdf". I should use the open_pdf function to read this document and then summarize it.

Action: open_pdf
Action Input: /Users/danid/Downloads/DARPA_grants_submission/DARPA-PS-25-02-BTO-01.pdfPDF Content from /Users/danid/Downloads/DARPA_grants_submission/DARPA-PS-25-02-BTO-01.pdf:

--- Page 1 ---
Special Topic: Ag x BTO
BACKGROUND:
The Defense Advanced Research Projects Agency (DARPA) is looking for innovative ideas to 
develop novel capabilities for national security. DARPA’s Biological Technologies Office (BTO) 
wishes to catalyze future efforts to defend agriculture against threats both naturally occurring 
and manmade.  Areas of interest include: (1) advances to early warning systems for chemical 
and biological threat surveillance and detection; (2) rapid-response agricultural 
countermeasures t

In [ ]:
# Run the agent, the "new way" with AgentExecutor. No change in usage pattern! just pass the input as a dict instead of a string. This is required to work with LangChain's new architecture.
agent_executor = AgentExecutor(agent=agent, tools=[open_pdf, browse_and_scrape], verbose=True, handle_parsing_errors=True)
response = agent_executor.invoke({"input": 
                                '''Open this URL page https://en.wikipedia.org/wiki/Patulin
                                 Browse and do basic scraping and find the IUPAC name of Patulin'''})
print(response["output"])




> Entering new AgentExecutor chain...

<think>I need to open the Wikipedia page about Patulin and find its IUPAC name. Let me use the browse_and_scrape function to access this URL and extract the main text content.

Action: browse_and_scrape
Action Input: https://en.wikipedia.org/wiki/Patulin</think>I'll help you find the IUPAC name of Patulin by browsing and scraping the Wikipedia page.

Thought: I need to open the Wikipedia page about Patulin using the browse_and_scrape function to access its content and find the IUPAC name.

Action: browse_and_scrape
Action Input: https://en.wikipedia.org/wiki/Patulin
Error accessing https://en.wikipedia.org/wiki/Patulin</think>I'll help you find the IUPAC name of Patulin by browsing and scraping the Wikipedia page.

Thought: I need to open the Wikipedia page about Patulin using the browse_and_scrape function to access its content and find the IUPAC name.

Action: browse_and_scrape
Action Input: https://en.wikipedia.org/wiki/Patulin
: 404 Client E

### Chat with the LLM Agent (instead of a single prompt)
The AgentExecutor by default is stateless (single-turn), but LangChain provides built-in memory components to make it stateful. Here's how to enable persistent conversations:

In [ ]:
from langchain.memory import ConversationBufferMemory # memory to store chat history
from langchain.prompts import ChatPromptTemplate, PromptTemplate, MessagesPlaceholder # prompt template with history placeholder
from langchain_core.messages.system import SystemMessage
from langchain_core.messages.human import HumanMessage

BASE_URL = "http://localhost:1234/v1"
API_KEY = "not-needed"  # Dummy key since LM Studio doesn't require auth

llm = ChatOpenAI(
    model_name=my_llms['gpt-oss'],  # Replace with your actual model name
    temperature=0.7,
    openai_api_key=API_KEY,
    openai_api_base=BASE_URL
)

# Initialize memory to store chat history
memory = ConversationBufferMemory(
    memory_key="chat_history",  # Stores history as "chat_history"
    return_messages=True, # Returns message objects (not just strings)
)

# Prompt template to use with agent of type "create_openai_tools_agent"
# prompt = hub.pull("hwchase17/openai-tools-agent")
# ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], ...

prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful assistant using provided tools."),
    MessagesPlaceholder(variable_name="chat_history"),  # History slot
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")  # For agent's internal steps
])

# Prompt template to use with agent of type "create_react_agent"
# prompt = hub.pull("hwchase17/react")
# PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], ...

# Create the agent with memory and prompt
agent = create_openai_tools_agent(
    llm=llm,
    tools=[browse_and_scrape, open_pdf, get_weather],
    prompt=prompt,  # includes history placeholder 
)

# Create the agent executor with memory
agent_executor = AgentExecutor(
    agent=agent, 
    tools=[open_pdf, browse_and_scrape, get_weather], 
    memory=memory,  # critical for history persistence 
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5)  # prevent infinite loops!)


# --- Conversation loop ---
response1 = agent_executor.invoke({"input": "What's the weather in Paris in August today? Assume that today is really August."})
response2 = agent_executor.invoke({"input": '''Can you compare it to Tokyo in the same month (i.e. August)?
                                    If you do not have API tools to access real-time weather data, base the answer on 
                                   your knowledge of historial weather data. But if you do, I would prefer to know the real-time data instead of historical.'''}) # Agent gets full context of previous exchange




> Entering new AgentExecutor chain...

The user is asking for the current weather in Paris for today, and they want me to assume that today is August. I should use the get_weather function with "Paris" as the location parameter to get the current weather information.
I'll check the current weather in Paris for you.
<tool_call>get_weather
<arg_key>location</arg_key>
<arg_value>Paris</arg_value>
</tool_call>

> Finished chain.


> Entering new AgentExecutor chain...

The user is asking me to compare Paris weather with Tokyo weather for August. They mention that if I don't have access to real-time data, I should use historical knowledge, but they prefer real-time data if available.

Looking at my tools, I have a get_weather function that can get current weather information for a location. Let me check what this tool actually does by looking at its description: "Get current weather information for a given location."

This suggests it provides real-time/current weather data, not historica

In [ ]:
def interactive_chat():
    '''Start an interactive chat session with the agent.
    Allows users to input messages and receive responses from the agent.
    Type 'exit' or 'quit' to end the session.
    '''
    while True:
        user_input = input("\nYou: ")
        
        if user_input.lower() in ["exit", "quit"]:
            break
            
        response = agent_executor.invoke({"input": user_input})
        print(f"\nAssistant: {response['output']}")

interactive_chat()

In [271]:
response3 = agent_executor.invoke({"input": ''' Can you browse and scrape this LangChain latest docs URL: https://python.langchain.com/docs/versions/migrating_memory/
                                    and summarize me the part about "Managing conversation history" - specifically what are the 
                                   differences between various Conversation Buffers object types'''})




> Entering new AgentExecutor chain...

Invoking: `browse_and_scrape` with `{'url': 'https://python.langchain.com/docs/versions/migrating_memory/'}`
responded: We need to browse the given URL and extract content, then locate section "Managing conversation history". Then summarize differences between various Conversation Buffers object types. Use browse_and_scrape tool.

Number of characters in text: 17699
Content from https://python.langchain.com/docs/versions/migrating_memory/:

Skip to main content Our Building Ambient Agents with LangGraph course is now available on LangChain Academy! Integrations API Reference More Contributing People Error reference LangSmith LangGraph LangChain Hub LangChain JS/TS v0.3 v0.3 v0.2 v0.1 💬 Search Introduction Tutorials Build a Question Answering application over a Graph Database Tutorials Build a simple LLM application with chat models and prompt templates Build a Chatbot Build a Retrieval Augmented Generation (RAG) App: Part 2 Build an Extraction C

In [265]:
response3 = agent_executor.invoke({"input": ''' Can you browse and scrape this GitHub page URL: https://github.com/steineggerlab/foldseek
                                    What is the main idea of this repository? What is it about?'''})




> Entering new AgentExecutor chain...

Invoking: `browse_and_scrape` with `{'url': 'https://github.com/steineggerlab/foldseek'}`
responded: Okay, let's see. The user wants me to browse and scrape the GitHub page at https://github.com/steineggerlab/foldseek. They're asking for the main idea and what the repository is about.

First, I need to check if the provided tools include a function for browsing and scraping. Looking at the tools, there's a browse_and_scrape function that takes a URL. Perfect, I should use that.

Wait, the user provided the exact URL, so I just need to call browse_and_scrape with that URL. Let me make sure the parameters are correct. The function requires "url" as a string, which they've given.

I should generate the tool call in the specified JSON format inside the tool_call tags. Let me structure that correctly. The name is browse_and_scrape, and the arguments should be {"url": "https://github.com/steineggerlab/foldseek"}.

I don't need to use any other tools s

In [269]:
response = agent_executor.invoke({"input": 
                                '''Open this URL page https://en.wikipedia.org/wiki/Patulin
                                    Browse and do basic scraping and find the IUPAC name of Patulin'''})




> Entering new AgentExecutor chain...

Invoking: `browse_and_scrape` with `{'url': 'https://en.wikipedia.org/wiki/Patulin'}`
responded: We need to browse the Wikipedia page and extract text, then locate the IUPAC name. Use function browse_and_scrape.

Number of characters in text: 14121
Content from https://en.wikipedia.org/wiki/Patulin:

Jump to content Main menu Main menu move to sidebar hide Navigation Main page Contents Current events Random article About Wikipedia Contact us Contribute Help Learn to edit Community portal Recent changes Upload file Special pages Search Search Appearance Donate Create account Log in Personal tools Donate Create account Log in Pages for logged out editors learn more Contributions Talk Contents move to sidebar hide (Top) 1 Biosynthesis, synthesis, and reactivity 2 Uses 3 Sources of exposure 4 Toxicity Toggle Toxicity subsection 4.1 Acute 4.2 Subacute 4.3 Genotoxicity 4.4 Reproduction studies 4.5 Immunotoxicity 4.6 Human health 5 Risk management and 

In [ ]:
print(response["output"])  # Print the first response

In [270]:
response = agent_executor.invoke({"input": 
                                '''Summarize the following PDF document at path:
                                /Users/danid/Downloads/DARPA_grants_submission/DARPA-PS-25-02-BTO-01.pdf
                                What are the main ideas behind the text? what is it about?'''})



> Entering new AgentExecutor chain...

Invoking: `open_pdf` with `{'file_path': '/Users/danid/Downloads/DARPA_grants_submission/DARPA-PS-25-02-BTO-01.pdf'}`
responded: We need to open the PDF file at given path. Use function open_pdf.

PDF Content from /Users/danid/Downloads/DARPA_grants_submission/DARPA-PS-25-02-BTO-01.pdf:

--- Page 1 ---
Special Topic: Ag x BTO
BACKGROUND:
The Defense Advanced Research Projects Agency (DARPA) is looking for innovative ideas to 
develop novel capabilities for national security. DARPA’s Biological Technologies Office (BTO) 
wishes to catalyze future efforts to defend agriculture against threats both naturally occurring 
and manmade.  Areas of interest include: (1) advances to early warning systems for chemical 
and biological threat surveillance and detection; (2) rapid-response agricultural 
countermeasures to defend against threats; (3) massively accelerated and expanded crop 
engineering for long-term threat defeat; (4) integrated and comprehensi

### Get weather info tool usage with LLM agent

In [182]:
from langchain_community.tools.openweathermap.tool import OpenWeatherMapQueryRun

# Set your OpenWeatherMap API key as an environment variable
os.environ["OPENWEATHERMAP_API_KEY"] = "a567d28529a5e1874a09941d6b58699e"

# Initialize the OpenWeatherMap tool
weather_tool = OpenWeatherMapQueryRun(
    name="weather",
    description="Get current weather information for a specified location"
)

# Define your custom tool wrapper
@tool("get_weather")
def get_weather(location: str) -> str:
    """
    Get current weather information for a given location.
    
    Args:
        location: Location name (e.g., "London", "New York")
        
    Returns:
        Formatted weather information
    """
    try:
        # Use the LangChain OpenWeatherMap tool internally
        result = weather_tool.invoke(location)
        return f"Weather in {location}: {result}"
    except Exception as e:
        return f"Error fetching weather: {str(e)}"


In [183]:
get_weather("Tel-Aviv")  # Example usage

'Weather in Tel-Aviv: In Tel-Aviv, the current weather is as follows:\nDetailed status: few clouds\nWind speed: 5.66 m/s, direction: 290°\nHumidity: 48%\nTemperature: \n  - Current: 31.27°C\n  - High: 32.88°C\n  - Low: 30.71°C\n  - Feels like: 32.67°C\nRain: {}\nHeat index: None\nCloud cover: 20%'

In [189]:
prompt = hub.pull("hwchase17/react")  # for create_react_agent()

# 3. Create the agent (REACT framework)
agent = create_react_agent(
    llm=llm,
    tools=[browse_and_scrape, open_pdf, get_weather],
    prompt=prompt,
)

# Run the agent, the "new way" with AgentExecutor. No change in usage pattern! just pass the input as a dict instead of a string. This is required to work with LangChain's new architecture.
agent_executor = AgentExecutor(agent=agent, tools=[open_pdf, browse_and_scrape, get_weather],
                               verbose=True,
                               handle_parsing_errors=True) # Avoids crashing on errors)

response = agent_executor.invoke({"input": 
                                '''What is the weather in Tel-Aviv today? Will it be comfortable to go out in the middle of the day?'''})
print(response["output"])


/opt/homebrew/lib/python3.12/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...

<think>I need to get weather information for Tel-Aviv today. I'll use the get_weather function with "Tel-Aviv" as the location parameter.</think>
Action: get_weather
Action Input: Tel-AvivWeather in Tel-Aviv: In Tel-Aviv, the current weather is as follows:
Detailed status: few clouds
Wind speed: 5.66 m/s, direction: 290°
Humidity: 48%
Temperature: 
  - Current: 31.27°C
  - High: 32.88°C
  - Low: 30.71°C
  - Feels like: 32.67°C
Rain: {}
Heat index: None
Cloud cover: 20%
<think>Based on the weather information, I can see that:
- Temperature is currently around 31.3°C (88°F) and will reach a high of about 32.9°C (91°F)
- There are few clouds with only 20% cloud cover
- Humidity is at 48%, which is moderate
- Wind speed is around 5.66 m/s (about 12.7 mph)

This information helps me determine if it will be comfortable to go out in the middle of the day. The temperature is quite high (close to 33°C or 91°F), which might be uncomfortable for many peopl